In [155]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [156]:
import csv
import re

def read_csv_file(file_path):
    # Initialisiere eine leere Liste, um die bereinigten Daten zu speichern
    cleaned_data = []

    # Lese die CSV-Datei
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Annahme: Semikolon als Trennzeichen
        for row in reader:
            # Verbinde die Zeilenelemente mit einem Komma, um das Trennzeichen zu vereinheitlichen
            unified_row = ','.join(row)
            # Ersetze mehrere aufeinander folgende Kommas durch ein einzelnes Komma
            unified_row = re.sub(r',+', ',', unified_row)
            # Teile die vereinheitlichte Zeile nach dem Komma auf
            split_row = unified_row.split(',')
            # Entferne die ersten zwei Parameter
            cleaned_row = split_row[1:]
            # Füge die bereinigte Zeile der Liste hinzu
            cleaned_data.append(cleaned_row)

    return cleaned_data

# Beispiel: Daten aus "input.csv" einlesen

In [157]:
data = pd.read_csv("initial_data.csv")
cleaned_data = read_csv_file("querys_ForcePush.csv")
df_queried = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
def add_data(data, queried_data):
    data = pd.concat([data, queried_data.iloc[:, :13]], axis=0)
    return data

print(df_queried.keys())

df_queried_2 = df_queried[df_queried['costs'] == str(1)]
data = add_data(data, df_queried_2)
print(len(data))
data = data.reset_index().astype('Float32')



#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'Pressure cylinder']]

print(len(x))
print(len(y))

x_max = x.max()
x_min = x.min()
y_max = y.max()
y_min = y.min()
y_var = y.var()

#print(str(y_min) + "\n:\n" + str(y_max))

y_range = y_max - y_min


Index(['Engine speed', 'Engine load', 'Railpressure', 'Air supply',
       'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature',
       'NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder', 'costs'],
      dtype='object')
137
137
137


In [158]:
# Datenaufteilung in Training und Test


print(len(x))
print(len(y))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

#X_train = scaler_X.fit_transform(X_train)
#X_test = scaler_X.transform(X_test)

#y_train = scaler_y.fit_transform(y_train)
#y_test = scaler_y.transform(y_test)


pls2 = PLSRegression(n_components=8)
pls2.fit(X_train, y_train)

test_results = pls2.predict(X_test)

print(test_results)
print(y_test)



137
137
[[2.45508588e+02 2.27741778e+00 5.29564330e+01 7.24162215e+01]
 [1.54622177e+02 2.09502163e+00 6.31165367e+01 9.13666873e+01]
 [1.27621110e+02 7.88495743e-01 1.99500371e+01 6.03266536e+01]
 [1.39798911e+02 1.50360447e+00 3.84033454e+01 6.32756700e+01]
 [2.08045927e+02 2.56732734e-01 1.90586978e+01 1.35074344e+02]
 [2.39847099e+02 2.83166976e+00 8.73350118e+01 9.91042952e+01]
 [7.64624700e+01 1.09840584e+00 2.73468419e+01 5.94353058e+01]
 [3.00396957e+02 1.40542613e+00 4.57397424e+01 1.19528518e+02]
 [4.38188825e+02 4.29456579e+00 1.22835402e+02 1.19074132e+02]
 [1.24584110e+02 1.98428365e+00 5.40721952e+01 6.67440000e+01]
 [4.40043626e+01 1.56520233e+00 3.94479608e+01 5.83720823e+01]
 [1.07568947e+02 1.41461740e+00 3.50063737e+01 5.99546205e+01]
 [4.53359122e+01 9.66659641e-01 2.12521773e+01 5.84488021e+01]
 [2.99527049e+02 3.18079710e+00 1.09838269e+02 1.23334864e+02]]
            NOx      PM 1         CO2  Pressure cylinder
105  267.957947  1.561381   48.390953          68.63

In [159]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error


featurewise_mse = []
featurewise_map = []
print(test_results.shape)
print(y_test.shape)
y_test = np.transpose(y_test)
test_results = np.transpose(test_results)
for feature_t, feature_p in zip(test_results, y_test.values):
    # print(feature_t)
    # print(feature_p)
    # print(mean_squared_error(feature_t, feature_p))
    featurewise_map.append(mean_absolute_percentage_error(feature_t, feature_p))
    featurewise_mse.append(mean_squared_error(feature_t, feature_p))

featurewise_mse = np.array(featurewise_mse)

featurewise_mape = np.array(featurewise_map)
#print(featurewise_mean)
# print(y_max.shape)
# print(featurewise_mean.shape)
# #print(y_range)
# featurewise_mean_percentage = featurewise_mean / y_max**2
# featurewise_mean_divided_by_variance = featurewise_mean / y_var
print("mean_squared_error:")
for mean in featurewise_mse:
    print('{:<10f}'.format(mean))
print("mean_absolute_percentage_error:")
for mean in featurewise_mape:
    print('{:<10f}'.format(mean*100))

print("\nAccuracy:")
for mape in featurewise_mape:
    print('{:<10f}'.format(100-mape*100))

# print("mse per feature in percentage to value range")
# print(featurewise_mean_percentage)
# print("featurewise_mean_divided_by_variance:")
# print(featurewise_mean_divided_by_variance)

(14, 4)
(14, 4)
mean_squared_error:
62041.241669
0.576458  
61.788895 
17.275330 
mean_absolute_percentage_error:
33.128966 
45.318696 
13.026932 
4.305052  

Accuracy:
66.871034 
54.681304 
86.973068 
95.694948 


In [160]:

submission_data = pd.read_csv("submission.csv")


#print(submission_data)
#print(submission_data.keys())

submission_keys = ['Speed', 'Load', 'Railpressure', 'Air', 'Angle', 'Pressure1','Pressure2', 'Temperature']
model_keys = ['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']

for sub_key, mod_key in zip(submission_keys, model_keys):
    submission_data = submission_data.rename(columns={sub_key:mod_key})



# input = input.reshape((1,8))
submission_result = pls2.predict(submission_data)


submission_result = np.transpose(submission_result)
submission_std = []
submission_mean = []
#print(submission_result)
for feature in submission_result:
    submission_std.append(feature.std())
    submission_mean.append(feature.mean())

print(submission_mean)
print(submission_std)

y_std = []
y_mean = []
#print(y)
y_t = np.transpose(np.array(y))

for feature in y_t:
    #print(feature)
    y_std.append(feature.std())
    y_mean.append(feature.mean())


print(y_mean)
print(y_std)

q_mean = np.divide(y_mean, submission_mean)
q_std = np.divide(y_std, submission_std)

print("\nQuotienten von mean und std:\n")
print(q_mean)
print(q_std)

# print(result)

[240.442048570784, 1.769951563317559, 54.48423493346019, 108.3416044721047]
[141.66732579230177, 1.0322483460031422, 32.81433817477528, 25.712357907840772]
[195.4656994325401, 1.8153307379162225, 55.88379297465303, 90.03473861026067]
[183.6805419907498, 1.4234482968418207, 39.56266604269441, 29.834454017263663]

Quotienten von mean und std:

[0.81294308 1.02563865 1.02568739 0.83102645]
[1.2965625  1.37897852 1.2056518  1.16031576]
